# Using the Unlabelled Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = pd.read_csv('data/data.csv')
target = pd.read_csv('data/target.csv')

In [3]:
new_data = pd.read_csv('data/new_data.csv')
new_target = pd.read_csv('data/new_target.csv')

# do feature scaling here

In [4]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
data = scaler.fit_transform(data)

# regression models

In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.cross_decomposition import PLSRegression

from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.metrics import mean_squared_error

In [6]:
from sklearn.model_selection import cross_val_score, KFold

# loop through the columns of target and find the best model for each column using data2 and target2. Do k-fold cross validation and take k as parameter to the function

def best_model2(X, y, k):

    models = [LinearRegression(), DecisionTreeRegressor(), RandomForestRegressor(), SVR(), KNeighborsRegressor(),
              GradientBoostingRegressor(), Ridge(), Lasso(), ElasticNet(), PLSRegression()]
    
    r2_scores = []
    rmse_scores = []

    for model in models:
        cv = KFold(n_splits=k, random_state=0, shuffle=True)
        r2_scores.append(np.mean(cross_val_score(model, X, y, cv=cv, scoring='r2')))
        rmse_scores.append(np.mean(np.sqrt(-cross_val_score(model, X, y, cv=cv, scoring='neg_mean_squared_error'))))

    model_scores = pd.DataFrame(sorted(list(zip([type(model).__name__ for model in models], r2_scores, rmse_scores)), key=lambda x: x[1], reverse=True), columns=['Model', 'R2 Score', 'RMSE'])

    return model_scores

# Using the unlabelled data

In [7]:
unlabelled_data = pd.read_csv('data/unlabelled.csv')
data3 = pd.read_csv('data/data.csv')
target3 = pd.read_csv('data/target.csv')

# Adding the batch experiment data

In [8]:
data3 = pd.concat([data3, new_data], axis=0)
target3 = pd.concat([target3, new_target], axis=0)

In [9]:
from scipy.signal import savgol_filter

# integrate the unlabelled data (data with no target) into the labelled data for better scaling and normalization
data3 = pd.concat([data3, unlabelled_data], axis=0)

# Baseline correction
data3 = data3 - data3.min()

# Normalization
data3 = data3 / data3.max()

# Noise reduction using smoothing techniques
data3 = savgol_filter(data3, 5, 2)
data3 = pd.DataFrame(data3, columns=unlabelled_data.columns)

# not doing pca because it only shows worse performance in the different models
# # Multiple scatter correction to reduce scattering effects
# data3 = pca.transform(data3)
# data3 = pd.DataFrame(data3)

# remove the unlabelled data from the complete data to get the labelled data
data3 = data3.iloc[:len(data3) - len(unlabelled_data)]

# loop through each column in target3 and find the best model using the best hyperparameters found by optuna


In [12]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Activation
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [14]:
import optuna

best_params = []

for column in target3.columns:
    X_train, X_test, y_train, y_test = train_test_split(data3, target3[column], test_size=0.2, random_state=0)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

    def objective(trial):
        model = Sequential()
        model.add(Dense(trial.suggest_int('units1', 32, 256), input_shape=(X_train.shape[1],)))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Dropout(trial.suggest_float('dropout1', 0.1, 0.5)))
        model.add(Dense(trial.suggest_int('units2', 32, 256)))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Dropout(trial.suggest_float('dropout2', 0.1, 0.5)))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')

        history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=32, verbose=0)

        return mean_squared_error(y_test, model.predict(X_test))

    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=100)

    trial = study.best_trial
    print(f'\n\n{column}:\n')
    print('MSE: ', trial.value)

    best_params.append(trial.params)

[I 2025-02-09 19:45:30,500] A new study created in memory with name: no-name-ec9c3e85-003c-4bfb-89d2-d45d101a94fd


2/2 [==============================] - 0s 3ms/step


[I 2025-02-09 19:45:35,472] Trial 0 finished with value: 261.3559385394974 and parameters: {'units1': 70, 'dropout1': 0.13545547316555448, 'units2': 173, 'dropout2': 0.22887818451245923}. Best is trial 0 with value: 261.3559385394974.


2/2 [==============================] - 0s 2ms/step


[I 2025-02-09 19:45:39,834] Trial 1 finished with value: 150.36273156228097 and parameters: {'units1': 40, 'dropout1': 0.1639233234678018, 'units2': 133, 'dropout2': 0.1356814934961966}. Best is trial 1 with value: 150.36273156228097.


...

2/2 [==============================] - 0s 3ms/step


[I 2025-02-09 20:30:25,534] Trial 98 finished with value: 0.35909473387632646 and parameters: {'units1': 190, 'dropout1': 0.2658953948604683, 'units2': 197, 'dropout2': 0.20319263756829123}. Best is trial 53 with value: 0.29862885766662084.


2/2 [==============================] - 0s 2ms/step


[I 2025-02-09 20:30:29,191] Trial 99 finished with value: 0.7436972793791153 and parameters: {'units1': 108, 'dropout1': 0.15946212767833226, 'units2': 171, 'dropout2': 0.18374318083557115}. Best is trial 53 with value: 0.29862885766662084.




Formate (g/L):

MSE:  0.29862885766662084


# loop through each column but this time use the best hyperparameters found by optuna and find the r2 score and rmse score

In [15]:
for i, column in enumerate(target3.columns):
    X_train, X_test, y_train, y_test = train_test_split(data3, target3[column], test_size=0.2, random_state=0)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

    model = Sequential()
    model.add(Dense(best_params[i]['units1'], input_shape=(X_train.shape[1],)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(best_params[i]['dropout1']))
    model.add(Dense(best_params[i]['units2']))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(best_params[i]['dropout2']))
    model.add(Dense(1))

    model.compile(optimizer='adam', loss='mean_squared_error')

    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=32, verbose=0)

    y_pred = model.predict(X_test)
    r2_score = model.evaluate(X_test, y_test)
    rmse_score = np.sqrt(mean_squared_error(y_test, y_pred))

    print(f'\n\n{column}:\n')
    print('R2 Score: ', r2_score)
    print('RMSE Score: ', rmse_score)

2/2 [==============================] - 0s 2ms/step - loss: 220.9439


Glucose (g/L):

R2 Score:  220.94390869140625
RMSE Score:  14.864182113133763
2/2 [==============================] - 0s 2ms/step - loss: 99.8784


Lactate (g/L):

R2 Score:  99.87838745117188
RMSE Score:  9.993917369126887
2/2 [==============================] - 0s 2ms/step - loss: 0.5069


Ethanol (g/L):

R2 Score:  0.5069400668144226
RMSE Score:  0.711997235207874
2/2 [==============================] - 0s 2ms/step - loss: 0.2747


Acetate (g/L):

R2 Score:  0.2746771574020386
RMSE Score:  0.5240965107828734
2/2 [==============================] - 0s 2ms/step - loss: 0.5177


Biomass (g/L):

R2 Score:  0.5176977515220642
RMSE Score:  0.7195121520944123
2/2 [==============================] - 0s 2ms/step - loss: 0.3894


Formate (g/L):

R2 Score:  0.3893965780735016
RMSE Score:  0.6240164945183301


Glucose (g/L):

>Test loss:  220.94390869140625 (not R2 score)
>
>RMSE Score:  14.864182113133763


Lactate (g/L):

>Test loss:  99.87838745117188
>
>RMSE Score:  9.993917369126887


Ethanol (g/L):

>Test loss:  0.5069400668144226
>
>RMSE Score:  0.711997235207874


Acetate (g/L):

>Test loss:  0.2746771574020386
>
>RMSE Score:  0.5240965107828734


Biomass (g/L):

>Test loss:  0.5176977515220642
>
>RMSE Score:  0.7195121520944123


Formate (g/L):

>Test loss:  0.3893965780735016
>
>RMSE Score:  0.6240164945183301

# also find the best regression model for each column in target3 using best_model2 function with k=5 (10 gives better results but takes longer time)

In [16]:
for column in target3.columns:
    model_scores = best_model2(data3, target3[column].values, 5)
    print(f'\n\n{column}:\n')
    print(model_scores)



Glucose (g/L):

                       Model  R2 Score       RMSE
0           LinearRegression  0.989870   1.576538
1  GradientBoostingRegressor  0.807923   6.606053
2      RandomForestRegressor  0.786442   7.060864
3      DecisionTreeRegressor  0.675527   8.568945
4        KNeighborsRegressor  0.650175   9.128605
5                      Ridge  0.646127   9.208028
6              PLSRegression  0.203671  13.927018
7                        SVR  0.087424  14.970300
8                 ElasticNet  0.030135  15.432048
9                      Lasso  0.008403  15.600416


Lactate (g/L):

                       Model  R2 Score       RMSE
0        KNeighborsRegressor  0.845121   6.642506
1  GradientBoostingRegressor  0.837059   6.975106
2      RandomForestRegressor  0.754432   8.266264
3                      Ridge  0.657631   9.889097
4      DecisionTreeRegressor  0.641189   9.473679
5              PLSRegression  0.341889  13.822119
6                 ElasticNet  0.168887  15.495018
7             

Glucose (g/L):

| Model                      | R2 Score | RMSE      |
|----------------------------|----------|-----------|
| LinearRegression           | 0.989870 | 1.576538  |
| GradientBoostingRegressor  | 0.807923 | 6.606053  |
| RandomForestRegressor      | 0.786442 | 7.060864  |
| DecisionTreeRegressor      | 0.675527 | 8.568945  |
| KNeighborsRegressor        | 0.650175 | 9.128605  |
| Ridge                      | 0.646127 | 9.208028  |
| PLSRegression              | 0.203671 | 13.927018 |
| SVR                        | 0.087424 | 14.970300 |
| ElasticNet                 | 0.030135 | 15.432048 |
| Lasso                      | 0.008403 | 15.600416 |

Lactate (g/L):

| Model                      | R2 Score | RMSE      |
|----------------------------|----------|-----------|
| KNeighborsRegressor        | 0.845121 | 6.642506  |
| GradientBoostingRegressor  | 0.837059 | 6.975106  |
| RandomForestRegressor      | 0.754432 | 8.266264  |
| Ridge                      | 0.657631 | 9.889097  |
| DecisionTreeRegressor      | 0.641189 | 9.473679  |
| PLSRegression              | 0.341889 | 13.822119 |
| ElasticNet                 | 0.168887 | 15.495018 |
| Lasso                      | 0.141296 | 15.747846 |
| SVR                        | 0.037384 | 16.735395 |
| LinearRegression           | -0.587507| 12.411790 |

Ethanol (g/L):

| Model                      | R2 Score | RMSE      |
|----------------------------|----------|-----------|
| LinearRegression           | 0.910407 | 0.215002  |
| GradientBoostingRegressor  | 0.533029 | 0.520427  |
| RandomForestRegressor      | 0.467411 | 0.535953  |
| KNeighborsRegressor        | 0.327890 | 0.603690  |
| Ridge                      | 0.325507 | 0.611733  |
| SVR                        | 0.119033 | 0.700774  |
| PLSRegression              | 0.112463 | 0.697178  |
| Lasso                      | -0.017500| 0.748614  |
| ElasticNet                 | -0.017500| 0.748614  |
| DecisionTreeRegressor      | -0.352318| 0.815760  |

Acetate (g/L):

| Model                      | R2 Score | RMSE      |
|----------------------------|----------|-----------|
| LinearRegression           | 0.682460 | 0.310424  |
| GradientBoostingRegressor  | 0.536231 | 0.390962  |
| RandomForestRegressor      | 0.501650 | 0.399880  |
| Ridge                      | 0.353852 | 0.451052  |
| KNeighborsRegressor        | 0.297449 | 0.462448  |
| DecisionTreeRegressor      | 0.200687 | 0.509511  |
| SVR                        | 0.188030 | 0.513491  |
| PLSRegression              | 0.135787 | 0.526222  |
| Lasso                      | -0.015361| 0.572668  |
| ElasticNet                 | -0.015361| 0.572668  |

Biomass (g/L):

| Model                      | R2 Score | RMSE      |
|----------------------------|----------|-----------|
| GradientBoostingRegressor  | 0.983416 | 0.241017  |
| RandomForestRegressor      | 0.980713 | 0.259718  |
| KNeighborsRegressor        | 0.966747 | 0.342507  |
| Ridge                      | 0.966628 | 0.335107  |
| SVR                        | 0.962287 | 0.351581  |
| DecisionTreeRegressor      | 0.959363 | 0.302704  |
| PLSRegression              | 0.804973 | 0.789552  |
| Lasso                      | -0.034224| 1.903718  |
| ElasticNet                 | -0.034224| 1.903718  |
| LinearRegression           | -6.710714| 2.668131  |

Formate (g/L):

| Model                      | R2 Score | RMSE      |
|----------------------------|----------|-----------|
| LinearRegression           | 0.675877 | 0.396016  |
| GradientBoostingRegressor  | 0.489027 | 0.488942  |
| RandomForestRegressor      | 0.464867 | 0.513867  |
| KNeighborsRegressor        | 0.348569 | 0.586573  |
| Ridge                      | 0.221717 | 0.631032  |
| SVR                        | 0.110807 | 0.696069  |
| PLSRegression              | 0.080158 | 0.702010  |
| Lasso                      | -0.021342| 0.745549  |
| ElasticNet                 | -0.021342| 0.745549  |
| DecisionTreeRegressor      | -0.362447| 0.746041  |

# also find the best regression model for each column in target3 using best_model2 function with k=10

In [17]:
for column in target3.columns:
    model_scores = best_model2(data3, target3[column].values, 10)
    print(f'\n\n{column}:\n')
    print(model_scores) 



Glucose (g/L):

                       Model  R2 Score       RMSE
0           LinearRegression  0.988769   1.614717
1  GradientBoostingRegressor  0.835508   5.956996
2      RandomForestRegressor  0.819535   6.462220
3                      Ridge  0.659568   8.873440
4        KNeighborsRegressor  0.658043   8.842925
5      DecisionTreeRegressor  0.616180   8.980659
6              PLSRegression  0.182083  13.869079
7                        SVR  0.069160  14.904967
8                 ElasticNet  0.002763  15.412115
9                      Lasso -0.020964  15.588674


Lactate (g/L):

                       Model  R2 Score       RMSE
0        KNeighborsRegressor  0.831301   6.232073
1  GradientBoostingRegressor  0.753150   6.752765
2      DecisionTreeRegressor  0.731441   7.367412
3      RandomForestRegressor  0.701033   7.616169
4                      Ridge  0.646520   9.622301
5              PLSRegression  0.295342  13.935056
6                 ElasticNet  0.132707  15.477187
7             

Glucose (g/L):

| Model                     | R2 Score | RMSE      |
|---------------------------|----------|-----------|
| LinearRegression          | 0.988769 | 1.614717  |
| GradientBoostingRegressor | 0.835508 | 5.956996  |
| RandomForestRegressor     | 0.819535 | 6.462220  |
| Ridge                     | 0.659568 | 8.873440  |
| KNeighborsRegressor       | 0.658043 | 8.842925  |
| DecisionTreeRegressor     | 0.616180 | 8.980659  |
| PLSRegression             | 0.182083 | 13.869079 |
| SVR                       | 0.069160 | 14.904967 |
| ElasticNet                | 0.002763 | 15.412115 |
| Lasso                     | -0.020964| 15.588674 |

Lactate (g/L):

| Model                     | R2 Score | RMSE      |
|---------------------------|----------|-----------|
| KNeighborsRegressor       | 0.831301 | 6.232073  |
| GradientBoostingRegressor | 0.753150 | 6.752765  |
| DecisionTreeRegressor     | 0.731441 | 7.367412  |
| RandomForestRegressor     | 0.701033 | 7.616169  |
| Ridge                     | 0.646520 | 9.622301  |
| PLSRegression             | 0.295342 | 13.935056 |
| ElasticNet                | 0.132707 | 15.477187 |
| Lasso                     | 0.105295 | 15.748981 |
| SVR                       | 0.034111 | 16.446900 |
| LinearRegression          | -6.711076| 32.972322 |

Ethanol (g/L):

| Model                     | R2 Score | RMSE      |
|---------------------------|----------|-----------|
| LinearRegression          | 0.890640 | 0.205750  |
| RandomForestRegressor     | 0.438678 | 0.508979  |
| GradientBoostingRegressor | 0.429386 | 0.504567  |
| KNeighborsRegressor       | 0.261753 | 0.565896  |
| Ridge                     | 0.256279 | 0.588913  |
| SVR                       | 0.115113 | 0.667215  |
| PLSRegression             | -0.008663| 0.680403  |
| Lasso                     | -0.089800| 0.726226  |
| ElasticNet                | -0.089800| 0.726226  |
| DecisionTreeRegressor     | -0.202997| 0.668237  |

Acetate (g/L):

| Model                     | R2 Score | RMSE      |
|---------------------------|----------|-----------|
| LinearRegression          | 0.672833 | 0.279081  |
| GradientBoostingRegressor | 0.557820 | 0.358910  |
| RandomForestRegressor     | 0.455993 | 0.367004  |
| Ridge                     | 0.242999 | 0.445187  |
| SVR                       | 0.196649 | 0.498876  |
| KNeighborsRegressor       | 0.162290 | 0.446007  |
| PLSRegression             | 0.086035 | 0.519341  |
| Lasso                     | -0.053566| 0.562975  |
| ElasticNet                | -0.053566| 0.562975  |
| DecisionTreeRegressor     | -0.512200| 0.492840  |

Biomass (g/L):

| Model                     | R2 Score | RMSE      |
|---------------------------|----------|-----------|
| GradientBoostingRegressor | 0.987698 | 0.202497  |
| RandomForestRegressor     | 0.982928 | 0.224319  |
| KNeighborsRegressor       | 0.974388 | 0.281367  |
| DecisionTreeRegressor     | 0.969533 | 0.319944  |
| Ridge                     | 0.968727 | 0.313383  |
| SVR                       | 0.966400 | 0.317420  |
| PLSRegression             | 0.746645 | 0.773859  |
| Lasso                     | -0.075516| 1.879509  |
| ElasticNet                | -0.075516| 1.879509  |
| LinearRegression          | -29.347372| 7.368810 |

Formate (g/L):

| Model                     | R2 Score | RMSE      |
|---------------------------|----------|-----------|
| GradientBoostingRegressor | 0.368196 | 0.427271  |
| LinearRegression          | 0.337277 | 0.395917  |
| RandomForestRegressor     | 0.243948 | 0.492314  |
| KNeighborsRegressor       | 0.231351 | 0.533445  |
| SVR                       | 0.138873 | 0.659297  |
| Ridge                     | -0.070047| 0.606121  |
| Lasso                     | -0.206286| 0.724714  |
| ElasticNet                | -0.206286| 0.724714  |
| PLSRegression             | -0.375717| 0.691459  |
| DecisionTreeRegressor     | -2.275494| 0.677532  |